# Demand Curve step by step

## Let's start with a survey in Google Forms about willingness to pay for four goods.  

We will create a few demand curves based on the class. Start by filling in the form at (also linked on data88e.org/fa25 or in Slide Deck)
 - https://forms.gle/vVwcy3mR5nt4br4W7   
 

In [ ]:
import pandas as pd
from datascience import *
import numpy as np
%matplotlib inline

## Find the Sheet ID in the URL of the Google Sheet!

*Behind every Google Form there is a Google Sheet*


Take a look at the data in the spreadsheet 


https://docs.google.com/spreadsheets/d/1HcRJzKtVfriF3cUpBgixYTsQC7--dhFq8hEJFt7l1BY/edit?usp=sharing

In [ ]:
sheet_id = "1HcRJzKtVfriF3cUpBgixYTsQC7--dhFq8hEJFt7l1BY"
sheet_name = "Form1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
# Last years data for comparison
#sheet_id = "1jp-XrFPk0eUNDUVWGa7Rmw9b0P8_jobTG0oLpvcHB9s"


Read it into a datascience table

In [ ]:
demand_table = Table.read_table(url)
demand_table

In [ ]:
demand_table.ihist("Coffee",bins=7)

In [ ]:
demand_table.ihist("Burrito",bins=7)

## Gonna roll with Burrito for this example

Step 1 - Lets pull out just Burritos 

This is a table with just Burrito prices that people are willing to pay ( bids)

In [ ]:
Burrito = demand_table.select("Burrito")
Burrito

Step 2 - Let's count the number at each price

And sort the table so that it is descending from high to low price


In [ ]:
# count the number at each price
Burrito_counts = demand_table.group("Burrito")
Burrito_counts = Burrito_counts.sort('Burrito', descending=True)

Burrito_counts

Step 3 - Let's pull out those counts 


In [ ]:
counts = Burrito_counts.column("count")
print(counts) 

Step 4 - use a numpy command called cumulative sum to get the number of people who will buy at each price


In [ ]:
cumulative_counts = counts.cumsum()
cumulative_counts

Step 5 - make an array of the prices of the burritos in descending order


In [ ]:
prices = make_array(20,17.5,15,12.5,10,7.5,5,2.5)
prices

Step 6 -  make a table with the prices and the cumulative counts


In [ ]:
demand_curve = Table().with_columns("Price", prices, "Cumulative Count", cumulative_counts)
demand_curve

In [ ]:
demand_curve.iscatter("Cumulative Count","Price")


In [ ]:
demand_curve.iscatter("Cumulative Count","Price", fit_line=True)


In [ ]:
# fit a line to the data using numpy        
m, b = np.polyfit(cumulative_counts,prices,  1)
print(m, b)

In [ ]:
# add a new column to the table with the log of price
demand_curve = demand_curve.with_column("Log Price", np.log(prices))
demand_curve

In [ ]:
#plot Q vs Log Price 
demand_curve.iscatter("Cumulative Count","Log Price", fit_line=True)


## New Section on Elasticities along the Demand Curve 


In [ ]:
demand_curve = demand_curve.relabel("Cumulative Count", "Quantity")
demand_curve

When we calculate elasticity, we need to compare how quantity changes as price changes. But our demand_curve table lists each price–quantity pair only once. To calculate a change, we need to line up each point with its neighbor on the demand curve:
	•	The first copy of the table gives us the “starting” price and quantity ($P_1, Q_1$).
	•	A shifted copy of the table (dropping the first row) gives us the “next” price and quantity ($P_2, Q_2$).

By putting these side by side, we can compute the differences:
	•	$\Delta Q = Q_2 - Q_1$
	•	$\Delta P = P_2 - P_1$

and then use the midpoint formula to find the elasticity for that segment of the demand curve.

To calculate elasticity, we’ll work with arrays from our original demand_curve table. First, we take out the arrays for prices and quantities. Then, one by one, we use those arrays to create new arrays: differences ($\Delta Q$, $\Delta P$), midpoints ($\bar Q$, $\bar P$), and the elasticity formulas. Finally, we stack all these arrays back together into a new elasticities table so we can see each segment of the demand curve with its calculated elasticity.

In [ ]:
# Make numpy arrays from the columns
P = demand_curve.column("Price")
Q = demand_curve.column("Quantity")
Q

In [ ]:
# Build shifted columns so we can line up pairs
# Take all rows except last
first_points = demand_curve.take(np.arange(demand_curve.num_rows - 1))
# Take all rows except first
second_points = demand_curve.take(np.arange(1, demand_curve.num_rows))
# Now we have a second table with the "Second Points   "
second_points
    

### We can make 4 arrays from which we can calculate the elasticities 

In [ ]:
# Make arrays from the columns of the two tables
P1 = first_points.column("Price")
P2 = second_points.column("Price")
Q1 = first_points.column("Quantity")
Q2 = second_points.column("Quantity")
Q2

### We can make the differences and the average 

In [ ]:
dQ = Q2 - Q1
Q_bar = (Q1 + Q2) / 2
dP = P2 - P1
P_bar = (P1 + P2) / 2
# print them
dQ, Q_bar, dP, P_bar

Reminder: the midpoint formula for elasticity

Elasticity measures the percent change in quantity relative to the percent change in price. To make sure the result doesn’t depend on which point we call “first” or “second,” we use the midpoint formula:
$$
E_d \;=\; \frac{\%\Delta Q}{\%\Delta P}
\;=\; \frac{\dfrac{Q_2 - Q_1}{\tfrac{Q_1 + Q_2}{2}}}{\dfrac{P_2 - P_1}{\tfrac{P_1 + P_2}{2}}}
$$

This formula uses the average of the two quantities ($\tfrac{Q_1+Q_2}{2}$) and the average of the two prices ($\tfrac{P_1+P_2}{2}$) as the bases for the percent changes.

That way, whether we move from $P_1$ to $P_2$ or from $P_2$ to $P_1$, the elasticity value comes out the same.

In [ ]:
pct_dQ = dQ / Q_bar
pct_dP = dP / P_bar
arc_E = pct_dQ / pct_dP
arc_E

Alternatively we can use the *log–difference elasticity formula* 

The definition of price elasticity of demand is:
$$
E_d = \frac{\%\Delta Q}{\%\Delta P}
$$
Using calculus, the “point” elasticity can be written as:
$$
E_d = \frac{dQ/Q}{dP/P} = \frac{d\ln Q}{d\ln P}.
$$
That means elasticity is the slope of $\ln(Q)$ with respect to $\ln(P)$.

When we only have data points (instead of continuous functions), we approximate the derivative with differences:
$$
E_d \;\approx\; \frac{\Delta \ln Q}{\Delta \ln P}
= \frac{\ln(Q_2) - \ln(Q_1)}{\ln(P_2) - \ln(P_1)}.
$$
This is exactly the log_E formula in our code. It’s another way to compute elasticity between two points, and it usually gives very similar results to the midpoint (arc) formula.

In [ ]:
log_E = (np.log(Q2) - np.log(Q1)) / (np.log(P2) - np.log(P1)) 
log_E 

### Let's make a new table with what we have 

In [ ]:
elasticities = Table().with_columns(
    "P1", P1,
    "P2", P2,
    "Q1", Q1,
    "Q2", Q2,
    "Midpoint Price", P_bar,
    "Midpoint Quantity", Q_bar,
    "Arc Elasticity", np.round(arc_E, 3),
    "LogDiff Elasticity", np.round(log_E, 3)
)

In [ ]:
elasticities.show()


### Visualizing elasticity

Once we’ve calculated elasticity values for each segment, it’s helpful to plot them against the midpoint price. This shows how demand tends to be more elastic at high prices (large negative elasticity) and more inelastic at low prices (elasticity close to 0).

We’ll add a horizontal line at $-1$ to mark the cutoff between elastic and inelastic demand.

In [ ]:
# Plot elasticity vs midpoint price
elasticities.iscatter("Midpoint Price", "Arc Elasticity")



In [ ]:
elasticities.iscatter("Midpoint Price", "Arc Elasticity", fit_line=True)